In [56]:
import pandas as pd
import numpy as  np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
# from sklearn.svm import SVC
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
import ast
from joblib import dump,load

In [23]:
import os
print(os.getcwd())

/mnt/c/Users/Ibinabo/Documents/DATA SCIENCE/WOMEN TECHSTER'S DATA SCIENCE AND AI FELLOWSHIP PROGRAM/PROJECTS/capstone_tec4dev


In [24]:
os.chdir("/mnt/c/Users/Ibinabo/Documents/DATA SCIENCE/WOMEN TECHSTER'S DATA SCIENCE AND AI FELLOWSHIP PROGRAM/PROJECTS/capstone_tec4dev")
print(os.getcwd())

/mnt/c/Users/Ibinabo/Documents/DATA SCIENCE/WOMEN TECHSTER'S DATA SCIENCE AND AI FELLOWSHIP PROGRAM/PROJECTS/capstone_tec4dev


In [25]:
filepath = "./DATA/psychometric_test_1000.csv"
df = pd.read_csv(filepath)

In [26]:
df.head()

,Mathematical Aptitude,Logical Reasoning,Openness,Conscientiousness,Emotional Stability,Agreeableness/ Assertiveness,Creativity,Aptitude,Programming,Design Software,3D Skills,Web Development Confidence,Suggested Tracks
0,Bad,Excellent,Somewhat Open,Not,Pressured (Composed),Cooperative,Creative,Technically Inclined,Intermediate,Basic,Moderate Experienced,Not Confident,"['Software Development', 'Blockchain']"
1,Average,Excellent,Not Open,Highly,Calm,Assertive,Creative,Technically Inclined,Advanced,Not Proficient,High Experienced,Highly Confident,"['3D Animation Skills', 'Product Design']"
2,Average,Average,Open,Not,Calm,Cooperative,Not Creative,Technically Inclined,Advanced,Highly Proficient,High Experienced,Not Confident,"['3D Animation Skills', 'Product Design']"
3,Bad,Excellent,Somewhat Open,Highly,Overwhelmed,Cooperative,Creative,Technically Inclined,Intermediate,Highly Proficient,Not Experienced,Not Confident,"['Cyber Security', 'Data Science']"
4,Excellent,Bad,Somewhat Open,Moderate,Calm,Cooperative,Creative,Not Technically Inclined,Intermediate,Basic,Moderate Experienced,Highly Confident,"['Software Development', 'Blockchain']"


In [27]:
df.shape

(1000, 13)

In [28]:
df2 = df

In [29]:
X = df2.drop(columns=["Suggested Tracks"])
y = df2["Suggested Tracks"]

In [30]:
print(y.head())

0       ['Software Development', 'Blockchain']
1    ['3D Animation Skills', 'Product Design']
2    ['3D Animation Skills', 'Product Design']
3           ['Cyber Security', 'Data Science']
4       ['Software Development', 'Blockchain']
Name: Suggested Tracks, dtype: object


In [31]:
from sklearn.preprocessing import LabelEncoder


# Assuming X is your DataFrame after label encoding
# Encode each categorical column using LabelEncoder
for col in X.columns:
    if X[col].dtype == object:  # Check for object type (categorical data)
        encoder = LabelEncoder()
        X[col] = encoder.fit_transform(X[col])
        
        #sanitize column name to remove spaces or slashes that could lead to error while saving
        safe_col_name = col.replace(' ','_').replace('/','_').replace('\\','_').replace('-','_')
        
        # Construct the file path using os.path.join for better compatibility
        file_path = os.path.join('DATA',f'{safe_col_name}_encoder.joblib')

        # save each encoder with sanitized name
        joblib.dump(encoder, file_path)
        
        # Get encoding mapping for the current column
        encoding_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
        print(f"Encoding mapping for {col}: {encoding_mapping}")


Encoding mapping for Mathematical Aptitude: {'Average': 0, 'Bad': 1, 'Excellent': 2}
Encoding mapping for Logical Reasoning: {'Average': 0, 'Bad': 1, 'Excellent': 2}
Encoding mapping for Openness: {'Not Open': 0, 'Open': 1, 'Somewhat Open': 2}
Encoding mapping for Conscientiousness: {'Highly': 0, 'Moderate': 1, 'Not': 2}
Encoding mapping for Emotional Stability: {'Calm': 0, 'Overwhelmed': 1, 'Pressured (Composed)': 2}
Encoding mapping for Agreeableness/ Assertiveness: {'Assertive': 0, 'Cooperative': 1}
Encoding mapping for Creativity: {'Creative': 0, 'Not Creative': 1}
Encoding mapping for Aptitude: {'Not Technically Inclined': 0, 'Technically Inclined': 1}
Encoding mapping for Programming: {'Advanced': 0, 'Basic': 1, 'Intermediate': 2}
Encoding mapping for Design Software: {'Basic': 0, 'Highly Proficient': 1, 'Not Proficient': 2, 'Proficient': 3}
Encoding mapping for 3D Skills: {'High Experienced': 0, 'Moderate Experienced': 1, 'Not Experienced': 2}
Encoding mapping for Web Developmen

In [32]:
X.head()

,Mathematical Aptitude,Logical Reasoning,Openness,Conscientiousness,Emotional Stability,Agreeableness/ Assertiveness,Creativity,Aptitude,Programming,Design Software,3D Skills,Web Development Confidence
0,1,2,2,2,2,1,0,1,2,0,1,2
1,0,2,0,0,0,0,0,1,0,2,0,0
2,0,0,1,2,0,1,1,1,0,1,0,2
3,1,2,2,0,1,1,0,1,2,1,2,2
4,2,1,2,1,0,1,0,0,2,0,1,0


In [33]:
# encoding y
# Convert each string representation to an actual list
y_lists = y.apply(ast.literal_eval).tolist()




In [34]:
# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Fit and transform the labels in 'y' to a binary matrix
y_encoded = mlb.fit_transform(y)

y_encoded = mlb.fit_transform(y_lists)

# Optionally, check the labels that have been encoded
print("Encoded Labels:", mlb.classes_)


Encoded Labels: ['3D Animation Skills' 'Blockchain' 'Cyber Security' 'Data Science'
 'Mobile App' 'Mobile Application' 'Product Design' 'Product Management'
 'Software Development']


In [35]:
# Define the file path for saving the MultiLabelBinarizer object
mlb_file_path = os.path.join('DATA', 'multi_label_binarizer.joblib')

# Save the MultiLabelBinarizer object for later use
joblib.dump(mlb, mlb_file_path)

print(f"MultiLabelBinarizer saved to {mlb_file_path}")




MultiLabelBinarizer saved to DATA/multi_label_binarizer.joblib


In [36]:
# To view the classes (unique labels) it has learned:
print("Classes found by MultiLabelBinarizer:", mlb.classes_)

Classes found by MultiLabelBinarizer: ['3D Animation Skills' 'Blockchain' 'Cyber Security' 'Data Science'
 'Mobile App' 'Mobile Application' 'Product Design' 'Product Management'
 'Software Development']


In [37]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
rf_predictions = rf_clf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Classifier Accuracy:", rf_accuracy)

Random Forest Classifier Accuracy: 0.985


In [39]:
rf_clf.score(X_train, y_train)

1.0

In [40]:
rf_clf.score(X_test, y_test)

0.985

In [41]:
print(classification_report(y_test, rf_predictions))

                                                precision    recall  f1-score   support

     ['3D Animation Skills', 'Product Design']       1.00      1.00      1.00        74
            ['Cyber Security', 'Data Science']       0.95      1.00      0.97        36
['Mobile Application', 'Software Development']       1.00      0.88      0.93         8
          ['Product Management', 'Mobile App']       1.00      0.86      0.92         7
        ['Software Development', 'Blockchain']       0.99      0.99      0.99        75

                                      accuracy                           0.98       200
                                     macro avg       0.99      0.94      0.96       200
                                  weighted avg       0.99      0.98      0.98       200



# Testing the model using a test data

In [42]:
# # Example prediction
# data =
# example = X.iloc[[0]]  # Taking the first row as an example
# predicted_tracks = rf_clf.predict(example)
# #predicted_tracks = predicted_tracks.reshape(-1, len(mlb.classes_))  # Reshape to (1, n_classes)
# #predicted_tracks = mlb.inverse_transform(predicted_tracks)
# print("Predicted Suggested Tracks:", predicted_tracks)

In [59]:
# # Sample data for one row
# Encoding mapping for Mathematical Aptitude: {'Average': 0, 'Bad': 1, 'Excellent': 2}
# Encoding mapping for Logical Reasoning: {'Average': 0, 'Bad': 1, 'Excellent': 2}
# Encoding mapping for Openness: {'Not Open': 0, 'Open': 1, 'Somewhat Open': 2}
# Encoding mapping for Conscientiousness: {'Highly': 0, 'Moderate': 1, 'Not': 2}
# Encoding mapping for Emotional Stability: {'Calm': 0, 'Overwhelmed': 1, 'Pressured (Composed)': 2}
# Encoding mapping for Agreeableness/ Assertiveness: {'Assertive': 0, 'Cooperative': 1}
# Encoding mapping for Creativity: {'Creative': 0, 'Not Creative': 1}
# Encoding mapping for Aptitude: {'Not Technically Inclined': 0, 'Technically Inclined': 1}
# Encoding mapping for Programming: {'Advanced': 0, 'Basic': 1, 'Intermediate': 2}
# Encoding mapping for Design Software: {'Basic': 0, 'Highly Proficient': 1, 'Not Proficient': 2, 'Proficient': 3}
# Encoding mapping for 3D Skills: {'High Experienced': 0, 'Moderate Experienced': 1, 'Not Experienced': 2}
# Encoding mapping for Web Development Confidence: {'Highly Confident': 0, 'Moderately Confident': 1, 'Not Confident': 2}

sample_data = {
    "Mathematical Aptitude": ["0"],
    "Logical Reasoning": ["2"],
    "Openness": ["1"],
    "Conscientiousness": ["0"],
    "Emotional Stability": ["0"],
    "Agreeableness/ Assertiveness": ["0"],
    "Creativity": ["0"],
    "Aptitude": ["1"],
    "Programming": ["2"],
    "Design Software": ["3"],
    "3D Skills": ["1"],
    "Web Development Confidence": ["1"]
}

# Convert to DataFrame
sample_df = pd.DataFrame(sample_data)

# Display the DataFrame
print(sample_df)

  Mathematical Aptitude Logical Reasoning Openness Conscientiousness  \
0                     0                 2        1                 0   

  Emotional Stability Agreeableness/ Assertiveness Creativity Aptitude  \
0                   0                            0          0        1   

  Programming Design Software 3D Skills Web Development Confidence  
0           2               3         1                          1  


In [44]:
# Example prediction

example = sample_df.iloc[[0]]  # Taking the first row as an example
predicted_tracks = rf_clf.predict(example)
#predicted_tracks = predicted_tracks.reshape(-1, len(mlb.classes_))  # Reshape to (1, n_classes)
#predicted_tracks = mlb.inverse_transform(predicted_tracks)
print("Predicted Suggested Tracks:", predicted_tracks)

Predicted Suggested Tracks: ["['Software Development', 'Blockchain']"]


# Hyper Parameter Tuning for Random Forest

In [45]:
# Random Forest Classifier hyperparameters grid
rf_param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10]}


In [46]:
rf_grid_search = GridSearchCV(RandomForestClassifier(), rf_param_grid, cv=5)
rf_grid_search.fit(X_train, y_train)
rf_best_model = rf_grid_search.best_estimator_
rf_predictions = rf_best_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Best Random Forest Model Parameters:", rf_grid_search.best_params_)
print("Best Random Forest Model Accuracy:", rf_accuracy)

Best Random Forest Model Parameters: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100}
Best Random Forest Model Accuracy: 0.99


In [47]:
print(confusion_matrix(y_test, rf_predictions))

[[74  0  0  0  0]
 [ 0 36  0  0  0]
 [ 0  0  7  0  1]
 [ 0  1  0  6  0]
 [ 0  0  0  0 75]]


In [48]:
print(classification_report(y_test, rf_predictions))

                                                precision    recall  f1-score   support

     ['3D Animation Skills', 'Product Design']       1.00      1.00      1.00        74
            ['Cyber Security', 'Data Science']       0.97      1.00      0.99        36
['Mobile Application', 'Software Development']       1.00      0.88      0.93         8
          ['Product Management', 'Mobile App']       1.00      0.86      0.92         7
        ['Software Development', 'Blockchain']       0.99      1.00      0.99        75

                                      accuracy                           0.99       200
                                     macro avg       0.99      0.95      0.97       200
                                  weighted avg       0.99      0.99      0.99       200



In [49]:
# Train Naive Bayes (Multinomial) Classifier
nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)
nb_predictions = nb_clf.predict(X_test)
nb_accuracy = accuracy_score(y_test, nb_predictions)
print("Naive Bayes (Multinomial) Classifier Accuracy:", nb_accuracy)

Naive Bayes (Multinomial) Classifier Accuracy: 0.51


In [50]:

# Naive Bayes (Multinomial) Classifier hyperparameters grid
nb_param_grid = {'alpha': [0.1, 1, 10], 'fit_prior': [True, False]}

In [51]:

# Perform GridSearchCV for Naive Bayes (Multinomial)
nb_grid_search = GridSearchCV(MultinomialNB(), nb_param_grid, cv=5)
nb_grid_search.fit(X_train, y_train)
nb_best_model = nb_grid_search.best_estimator_
nb_predictions = nb_best_model.predict(X_test)
nb_accuracy = accuracy_score(y_test, nb_predictions)
print("Best Naive Bayes (Multinomial) Model Parameters:", nb_grid_search.best_params_)
print("Best Naive Bayes (Multinomial) Model Accuracy:", nb_accuracy)

Best Naive Bayes (Multinomial) Model Parameters: {'alpha': 10, 'fit_prior': True}
Best Naive Bayes (Multinomial) Model Accuracy: 0.515


## ☄ **Model Deplyment**

---

Before deployment, the best model (Random Forest) will be serialized (saved) for export and deployment.

In [57]:
#serializing and saving the best model

# Construct the path to the file
file_path = os.path.join('DATA', 'rf_best_model.joblib')

dump(rf_best_model, file_path)

['DATA/rf_best_model.joblib']

In [ ]:
#to load the model

loaded_model = load('rf_best_model.joblib')